# Neural Network

## Neural Networks

Neural Networks are a machine learning framework that attempts to mimic the learning pattern of natural biological neural networks. Biological neural networks have interconnected neurons with dendrites that receive inputs, then based on these inputs they produce an output signal through an axon to another neuron. We will try to mimic this process through the use of Artificial Neural Networks (ANN), which we will just refer to as neural networks from now on. The process of creating a neural network begins with the most basic form, a single perceptron. Let's start by explaining the single perceptron!

## The Perceptron

Let's start our discussion by talking about the Perceptron! A perceptron has one or more inputs, a bias, an activation function, and a single output. The perceptron receives inputs, multiplies them by some weight, and then passes them into an activation function to produce an output. There are many possible activation functions to choose from, such as the logistic function, a trigonometric function, a step function etc. We also make sure to add a bias to the perceptron, this avoids issues where all inputs could be equal to zero (meaning no multiplicative weight would have an effect).

Once we have the output we can compare it to a known label and adjust the weights accordingly (the weights usually start off with random initialization values). We keep repeating this process until we have reached a maximum number of allowed iterations, or an acceptable error rate.

To create a neural network, we simply begin to add layers of perceptrons together, creating a multi-layer perceptron model of a neural network. You'll have an input layer which directly takes in your feature inputs and an output layer which will create the resulting outputs. Any layers in between are known as hidden layers because they don't directly "see" the feature inputs or outputs.

Keep in mind that due to their nature, neural networks tend to work better on GPU than on CPU. The sci-kit learn framework isn't built for GPU optimization. If you want to continue to GPU and distributed models, take a look at some other frameworks, such as Google's open sourced TensforFlow.

Let's move on to actually creating a neural network with Python and Sci-Kit Learn!


## Data

Improve on the state of the art in credit scoring by predicting the probability that somebody will experience financial distress in the next two years.

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv('data.csv')

dfEduInstituteGroup = pd.get_dummies(df['Max_EduInstituteGroup'], prefix='Max_EduInstituteGroup')
dfHireTypeGroup = pd.get_dummies(df['HireType'], prefix='HireType')
dfHireSourceGroup = pd.get_dummies(df['HireSourceGroup'], prefix='HireSourceGroup')

#df = df.drop(['EduInstituteGroup','HireTypeGroup','HireSourceGroup'], axis=1)

df = pd.concat([df, dfEduInstituteGroup,dfHireTypeGroup,dfHireSourceGroup], axis=1)


In [25]:
print(df.shape)
df.head()

(4591, 40)


,SerialNumber,Leave,ActionYear,WorkDurationYear,CountLoan,Avg_MonthPerLoan,HireType,HireSourceGroup,WorkDurationYear.1,Avg_TotalAbsensePerYear,...,HireType_Experienced Hire,HireType_Inexperienced Hire,HireType_Unknown,HireSourceGroup_Agency,HireSourceGroup_Campus/Fair,HireSourceGroup_Contractor Conversion,HireSourceGroup_Other,HireSourceGroup_Referral,HireSourceGroup_Unknown,HireSourceGroup_Website/Ads
0,4,1.0,2000,39.0,0.0,0.0,Unknown,Unknown,39.0,0.0,...,0,0,1,0,0,0,0,0,1,0
1,5,1.0,2000,39.0,0.0,0.0,Unknown,Unknown,39.0,0.0,...,0,0,1,0,0,0,0,0,1,0
2,6,1.0,2000,38.0,0.0,0.0,Unknown,Unknown,38.0,0.0,...,0,0,1,0,0,0,0,0,1,0
3,7,1.0,2000,38.0,0.0,0.0,Unknown,Unknown,38.0,0.0,...,0,0,1,0,0,0,0,0,1,0
4,10,1.0,2000,38.0,0.0,0.0,Unknown,Unknown,38.0,0.0,...,0,0,1,0,0,0,0,0,1,0


Let's check out the data:

In [26]:
df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
SerialNumber,4591.0,2685.561316,1408.065369,4.0,1552.5,2743.000000,3890.500000,5038.0
Leave,4591.0,0.517752,0.499739,0.0,0.0,1.000000,1.000000,1.0
ActionYear,4591.0,2012.212590,5.162427,2000.0,2008.0,2014.000000,2016.000000,2017.0
WorkDurationYear,4591.0,9.029841,9.515552,0.0,2.0,6.000000,13.000000,42.0
CountLoan,4591.0,0.337617,0.561412,0.0,0.0,0.000000,1.000000,3.0
Avg_MonthPerLoan,4591.0,17.570500,35.784636,0.0,0.0,0.000000,15.000000,174.0
WorkDurationYear.1,4591.0,9.029841,9.515552,0.0,2.0,6.000000,13.000000,42.0
Avg_TotalAbsensePerYear,4591.0,4.031373,5.392830,0.0,0.0,3.100000,6.652174,216.0
Avg_NumDaysPerAbsense,4591.0,1.194582,1.720895,0.0,0.0,1.196078,1.562500,45.0
TotalEduAllowance,4591.0,0.361141,1.116057,0.0,0.0,0.000000,0.000000,12.0


In [27]:
df.shape

(4591, 40)

### Train Test Split

Let's split our data into training and testing sets, this is done easily with SciKit Learn's train_test_split function from model_selection:

In [29]:
from sklearn.model_selection import train_test_split

df_train = df[ df['ActionYear'] < 2017]
df_train.shape

df_test = df[ df['ActionYear'] >= 2017]
df_test.shape

(1122, 40)

In [0]:
df_train_variable = df_train.drop(['SerialNumber','ActionYear','Leave','Max_EduInstituteGroup','HireType','HireSourceGroup'],axis=1)
df_train_label = df_train['Leave']

df_test_variable = df_test.drop(['SerialNumber','ActionYear','Leave','Max_EduInstituteGroup','HireType','HireSourceGroup'],axis=1)
df_test_label = df_test['Leave']



#X_train, X_test, y_train, y_test = train_test_split(df_variable, df_label, test_size=0.30, random_state=101)
X_train, X_test, y_train, y_test = df_train_variable, df_test_variable, df_train_label, df_test_label


## Data Preprocessing

The neural network may have difficulty converging before the maximum number of iterations allowed if the data is not normalized. Multi-layer Perceptron is sensitive to feature scaling, so it is highly recommended to scale your data. Note that you must apply the same scaling to the test set for meaningful results.  There are a lot of different methods for normalization of data, we will use the built-in StandardScaler for standardization.

In [0]:
from sklearn.preprocessing import StandardScaler

In [0]:
scaler = StandardScaler()

In [33]:
# Fit only to the training data
scaler.fit(X_train)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [0]:
# Now apply the transformations to the data:
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

## Training the model

Now it is time to train our model. SciKit Learn makes this incredibly easy, by using estimator objects. In this case we will import our estimator (the Multi-Layer Perceptron Classifier model) from the neural_network library of SciKit-Learn!

In [0]:
from sklearn.neural_network import MLPClassifier

In [0]:
mlp = MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=True, epsilon=1e-08,
       hidden_layer_sizes=22, learning_rate='constant',
       learning_rate_init=0.001, max_iter=500, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=12345,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.2,
       verbose=False, warm_start=False)

In [37]:
mlp.fit(X_train, y_train)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=True, epsilon=1e-08,
       hidden_layer_sizes=22, learning_rate='constant',
       learning_rate_init=0.001, max_iter=500, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=12345,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.2,
       verbose=False, warm_start=False)

You can see the output that shows the default values of the other parameters in the model. I encourage you to play around with them and discover what effects they have on your model!

## Predictions and Evaluation

Now that we have a model it is time to use it to get predictions! We can do this simply with the predict() method off of our fitted model:

In [0]:
predictions = mlp.predict(X_test)

Now we can use SciKit-Learn's built in metrics such as a classification report and confusion matrix to evaluate how well our model performed:

In [0]:
from sklearn.metrics import classification_report, confusion_matrix

In [40]:
print(confusion_matrix(y_test,predictions))

[[471 488]
 [ 59 104]]


In [41]:
print(classification_report(y_test,predictions))

             precision    recall  f1-score   support

        0.0       0.89      0.49      0.63       959
        1.0       0.18      0.64      0.28       163

avg / total       0.79      0.51      0.58      1122



Not bad! Looks like we only miss-classified one bottle of wine in our test data! This is pretty good considering how few lines of code we had to write. The downside however to using a Multi-Layer Preceptron model is how difficult it is to interpret the model itself. The weights and biases won't be easily interpretable in relation to which features are important to the model itself.

However, if you do want to extract the MLP weights and biases after training your model, you use its public attributes **coefs_** and **intercepts_**. 

**coefs_** is a list of weight matrices, where weight matrix at index i represents the weights between layer i and layer i+1. 

**intercepts_** is a list of bias vectors, where the vector at index i represents the bias values added to layer i+1.

In [42]:
len(mlp.coefs_)

2

In [43]:
len(mlp.coefs_[0])

34

In [44]:
len(mlp.intercepts_[0])

22

In [45]:
from sklearn.pipeline import Pipeline

# scaler = StandardScaler()
# mlp = MLPClassifier()

estimators = [('scaler', scaler), ('mlp', mlp)]

pipe = Pipeline(estimators)
pipe

Pipeline(memory=None,
     steps=[('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('mlp', MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=True, epsilon=1e-08,
       hidden_layer_sizes=22, learning_rate='constant',
       learning_rate_ini...=True, solver='adam', tol=0.0001, validation_fraction=0.2,
       verbose=False, warm_start=False))])

In [46]:
pipe.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('mlp', MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=True, epsilon=1e-08,
       hidden_layer_sizes=22, learning_rate='constant',
       learning_rate_ini...=True, solver='adam', tol=0.0001, validation_fraction=0.2,
       verbose=False, warm_start=False))])

In [47]:
pipe.score(X_test, y_test)

0.5124777183600713